# Intro

> Generate [object_per_actor_nmvw.csv](object_per_actor_nmvw.csv)

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT (COUNT(DISTINCT ?nmvw_object) AS ?no_object) ?nmvw_actor
WHERE{
  GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
  {?nmvw_acq crm:P24_transferred_title_of ?nmvw_object .
   ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor .}
  UNION
  {
    ?nmvw_prod crm:P14_carried_out_by ?nmvw_actor .
    ?nmvw_object crm:P108i_was_produced_by ?nmvw_prod .
  }
  ?nmvw_object a crm:E22_Human-Made_Object .
  }
}GROUP BY ?nmvw_actor
```

In [4]:
def cleanup(x: str):
    return int(str(x).replace('"', '').strip())

import pandas as pd

df1 = pd.read_csv("object_per_actor_nmvw.csv")
df1 = df1.iloc[: , :-1]
df1.columns = ['no_object', 'nmvw_actor']
df1['no_object'] = df1['no_object'].apply(cleanup)
df1 = df1.sort_values(by=['no_object'], ascending=False)
df1

,no_object,nmvw_actor
4564,135424,"""https://hdl.handle.net/20.500.11840/pi4332"""
4048,69382,"""https://hdl.handle.net/20.500.11840/pi40292"""
2608,13609,"""https://hdl.handle.net/20.500.11840/pi3358"""
1448,9729,"""https://hdl.handle.net/20.500.11840/pi2246"""
7559,7464,"""https://hdl.handle.net/20.500.11840/pi57572"""
...,...,...
2558,1,"""https://hdl.handle.net/20.500.11840/pi32984"""
2560,1,"""https://hdl.handle.net/20.500.11840/pi3300"""
2561,1,"""https://hdl.handle.net/20.500.11840/pi3305"""
7653,1,"""https://hdl.handle.net/20.500.11840/pi57704"""


> Generate [object_per_actor_bb.csv](object_per_actor_bb.csv)
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT (COUNT(DISTINCT ?bb_obj) AS ?no_object) ?bb_actor
WHERE{
    {?bb_acq crm:P23_transferred_title_from ?bb_actor .	
    ?bb_acq crm:P24_transferred_title_of ?bb_obj .}
    UNION
    {
          ?bb_obj crm:P51_has_former_or_current_owner ?bb_actor .
    }

GRAPH <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
    ?bb_obj a crm:E22_Human-Made_Object. }
}GROUP BY ?bb_actor
```
> Note: https://pressingmatter.nl/Bronbeek/Constituents/1896" --> "Koninklijk Tehuis voor Oud-Militairen en Museum Bronbeek"^^xsd:string
> https://pressingmatter.nl/Bronbeek/Constituents/9806 --> Nijmeegs Volkenkundig Museum
> https://pressingmatter.nl/Bronbeek/Constituents/9804 --> "Gemeente Nijmegen"
> https://pressingmatter.nl/Bronbeek/Constituents/9805 --> Jean Louis Henri Beijens

In [2]:
def cleanup(x: str):
    return int(str(x).replace('"', '').strip())

import pandas as pd

df1 = pd.read_csv("object_per_actor_bb.csv")
df1 = df1.iloc[: , :-1]
df1.columns = ['no_object', 'bb_actor']
df1['no_object'] = df1['no_object'].apply(cleanup)
df1 = df1.sort_values(by=['no_object'], ascending=False)
df1

,no_object,bb_actor
2663,12409,"""https://pressingmatter.nl/Bronbeek/Constitue..."
6998,3066,"""https://pressingmatter.nl/Bronbeek/Constitue..."
6996,3066,"""https://pressingmatter.nl/Bronbeek/Constitue..."
6997,3066,"""https://pressingmatter.nl/Bronbeek/Constitue..."
4227,1542,"""https://pressingmatter.nl/Bronbeek/Constitue..."
...,...,...
5181,1,"""https://pressingmatter.nl/Bronbeek/Constitue..."
5182,1,"""https://pressingmatter.nl/Bronbeek/Constitue..."
5183,1,"""https://pressingmatter.nl/Bronbeek/Constitue..."
2343,1,"""https://pressingmatter.nl/Bronbeek/Constitue..."


# CQ-1a

How many actors we have with one more objects and also have military background?

SPARQL Query:
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT (COUNT(DISTINCT ?nmvw_actor) AS ?no_actor)
WHERE{
  GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
  {?nmvw_acq crm:P24_transferred_title_of ?nmvw_object .
   ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor .}
  UNION
  {
    ?nmvw_prod crm:P14_carried_out_by ?nmvw_actor .
    ?nmvw_object crm:P108i_was_produced_by ?nmvw_prod .
  }
  ?nmvw_object a crm:E22_Human-Made_Object .
  }
  ?nmvw_actor owl:sameAs ?bb_actor .
}
```

> Note that, here we consider `production actor` too! 

1. **Initial+Surname Match**: 277 actor from NMVW dataset, has connection with at least one object's provenance from NMVW dataset and also have owl:sameAS link with Bronbeek actor.
2. **Exact String Matching**: 117 actor from NMVW dataset, has connection with at least one object's provenance from NMVW dataset and also have owl:sameAS link with Bronbeek actor.
3. **Surname Matching**: 2680 actor from NMVW dataset, has connection with at least one object's provenance from NMVW dataset and also have owl:sameAS link with Bronbeek actor.
4. **Fuzzy String Matching**: 6779 actor from NMVW dataset, has connection with at least one object's provenance from NMVW dataset and also have owl:sameAS link with Bronbeek actor.
5. **Deezy Match**: 87 actor from NMVW dataset, has connection with at least one object's provenance from NMVW dataset and also have owl:sameAS link with Bronbeek actor.


# CQ 1b

Query to generate [CQ1b_nmvw_obj.csv](CQ1b_nmvw_obj.csv)
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?nmvw_object ?nmvw_actor
WHERE{
  GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
  {?nmvw_acq crm:P24_transferred_title_of ?nmvw_object .
   ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor .}
  UNION
  {
    ?nmvw_prod crm:P14_carried_out_by ?nmvw_actor .
    ?nmvw_object crm:P108i_was_produced_by ?nmvw_prod .
  }
  ?nmvw_object a crm:E22_Human-Made_Object .
  }
  # removing "onbeked / unknown"
  FILTER (?nmvw_actor != <https://hdl.handle.net/20.500.11840/pi4332>)
}
```

In [88]:
def cleanup(x: str):
    return int(str(x).replace('"', '').strip())

import pandas as pd

df1 = pd.read_csv("CQ1b_nmvw_obj.csv")
df1 = df1.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
df1 = df1.iloc[: , :-1]
df1.columns = ['nmvw_object', 'nmvw_actor']
df1

,nmvw_object,nmvw_actor
0,https://hdl.handle.net/20.500.11840/128796,https://hdl.handle.net/20.500.11840/pi1125
1,https://hdl.handle.net/20.500.11840/22185,https://hdl.handle.net/20.500.11840/pi39586
2,https://hdl.handle.net/20.500.11840/22199,https://hdl.handle.net/20.500.11840/pi7008
3,https://hdl.handle.net/20.500.11840/22199,https://hdl.handle.net/20.500.11840/pi39586
4,https://hdl.handle.net/20.500.11840/22201,https://hdl.handle.net/20.500.11840/pi2354
...,...,...
621439,https://hdl.handle.net/20.500.11840/277225,https://hdl.handle.net/20.500.11840/pi44822
621440,https://hdl.handle.net/20.500.11840/277226,https://hdl.handle.net/20.500.11840/pi2687
621441,https://hdl.handle.net/20.500.11840/277227,https://hdl.handle.net/20.500.11840/pi2246
621442,https://hdl.handle.net/20.500.11840/277228,https://hdl.handle.net/20.500.11840/pi2246


In [89]:
# Group by 'nmvw_actor' and count unique 'nmvw_object' per group
unique_counts = df1.groupby('nmvw_actor')['nmvw_object'].nunique()

# Filter rows where 'nmvw_actor' has less than 2 unique 'nmvw_object'
filtered_df = df1[df1['nmvw_actor'].isin(unique_counts[unique_counts > 1].index)]

print(filtered_df['nmvw_object'].nunique())

444516


> out of 446,310 unique objects who have connection with actor, 444,517 can be linked with at least one other objects

Generate [CQ1b_bb_obj.csv](CQ1b_bb_obj.csv)

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?nmvw_actor (COUNT(DISTINCT ?bb_object) AS ?n_bb_object) 
WHERE{
    ?nmvw_actor owl:sameAs ?bb_actor .
    #acquisition event
    {
    	?bb_acq crm:P23_transferred_title_from ?bb_actor .   
    	?bb_acq crm:P24_transferred_title_of ?bb_object .
    }
  UNION
    {
      	?bb_object crm:P51_has_former_or_current_owner ?bb_actor .
    }
  GRAPH <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
    ?bb_object a crm:E22_Human-Made_Object .
    }
} GROUP BY ?nmvw_actor
```

In [90]:
def cleanup(x: str):
    return int(str(x).replace('"', '').strip())

import pandas as pd

df2 = pd.read_csv("CQ1b_bb_obj.csv")
df2 = df2.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
# df2 = df2.iloc[: , :-1]
df2.columns = ["nmvw_actor", "bb_object" ]
df2['bb_object'] = df2['bb_object'].apply(cleanup)

df2 = df2[df2['nmvw_actor'].isin(df1['nmvw_actor'])]

df2["bb_object"].sum()

310

In [91]:
print(f"The number of actor contributing to this object count is: {df2['nmvw_actor'].nunique()}")

The number of actor contributing to this object count is: 36


> does any of the actor who have only one object, now have more than one connected object due to the connection of bronbeek,

In [4]:
df2[df2['nmvw_actor'].isin(unique_counts[unique_counts == 1].index)]['nmvw_actor'].nunique()

15

In [6]:
310 + 15

325

# CQ 1c

How many possible object to object connection we have, by establishing links via actor?

Generate [CQ1b_nmvw_obj.csv](CQ1b_nmvw_obj.csv)

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT (COUNT(DISTINCT ?nmvw_object) AS ?n_nmvw_object) ?nmvw_actor
WHERE{
  GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
  {?nmvw_acq crm:P24_transferred_title_of ?nmvw_object .
   ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor .}
  UNION
  {
    ?nmvw_prod crm:P14_carried_out_by ?nmvw_actor .
    ?nmvw_object crm:P108i_was_produced_by ?nmvw_prod .
  }
  ?nmvw_object a crm:E22_Human-Made_Object .
  }
  # removing "onbeked / unknown"
  FILTER (?nmvw_actor != <https://hdl.handle.net/20.500.11840/pi4332>)
}GROUP BY ?nmvw_actor
```

> Note: https://hdl.handle.net/20.500.11840/pi40292 / "Stichting Indisch Wetenschappelijk Instituut" generate 69382 object connection but keeping this for now. 

In [ ]:
def cleanup(x: str):
    return int(str(x).replace('"', '').strip())

import math

def calculate_combinations(total_objects):
    combination_value = math.factorial(total_objects) // (math.factorial(2) * math.factorial(total_objects - 2))
    return combination_value

import pandas as pd

df1 = pd.read_csv("CQ1b_nmvw_obj.csv")
df1 = df1.iloc[: , :-1]
df1.columns = ['n_nmvw_object', 'nmvw_actor']
df1['n_nmvw_object'] = df1['n_nmvw_object'].apply(cleanup)
df1 = df1[df1.n_nmvw_object != 1]
df1 = df1.sort_values(by=['n_nmvw_object'], ascending=False)
df1['possible_connections'] = df1['n_nmvw_object'].apply(calculate_combinations)
df1['possible_connections'].sum()


In [ ]:
df1

How many object to object connection we have, by establishing links via actor when we also add objects from bronbeek via the owl:sameAs connection? 

Generate [CQ1b_bb_obj.csv](CQ1b_bb_obj.csv)
SPARQL QUERY:
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?nmvw_actor (COUNT(DISTINCT ?bb_object) AS ?n_bb_object) 
WHERE{
    ?nmvw_actor owl:sameAs ?bb_actor .
    #acquisition event
    {
    	?bb_acq crm:P23_transferred_title_from ?bb_actor .   
    	?bb_acq crm:P24_transferred_title_of ?bb_object .
    }
  UNION
    {
      	?bb_object crm:P51_has_former_or_current_owner ?bb_actor .
    }
  GRAPH <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
    ?bb_object a crm:E22_Human-Made_Object .
    }
} GROUP BY ?nmvw_actor

```

In [ ]:
import pandas as pd

def cleanup(x: str):
    return int(str(x).replace('"', '').strip())

import math

def calculate_combinations(total_objects):
    combination_value = math.factorial(total_objects) // (math.factorial(2) * math.factorial(total_objects - 2))
    return combination_value

import pandas as pd

df2 = pd.read_csv("CQ1b_bb_obj.csv")
df2 = df2.iloc[: , :-1]
df2.columns = ["nmvw_actor" , "n_bb_object"]
df2['n_bb_object'] = df2['n_bb_object'].apply(cleanup)
df2 = df2.sort_values(by=['n_bb_object'], ascending=False)
df2


In [ ]:
# Merge the dataframes on 'nmvw_actor'
merged_df = pd.merge(df1, df2, on='nmvw_actor', how='left')
merged_df['n_bb_object'] = merged_df['n_bb_object'].fillna(0)

# Add the 'n_nmvw_object' and 'n_bb_object' columns where 'nmvw_actor' matches
merged_df['combined_values'] = merged_df['n_nmvw_object'] + merged_df['n_bb_object']

merged_df['possible_connections'] = merged_df['combined_values'].apply(calculate_combinations)
merged_df['possible_connections'].sum()

# CQ 2

SPARQL Query:
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT (COUNT(DISTINCT ?nmvw_object) AS ?n)
WHERE{
  Graph <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
    {
     ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor .
     ?nmvw_acq crm:P24_transferred_title_of ?nmvw_object .
    }
    UNION
    {
      ?nmvw_object crm:P108i_was_produced_by ?nmvw_prod .
      ?nmvw_prod crm:P14_carried_out_by ?nmvw_actor .
    }
    ?nmvw_object  a crm:E22_Human-Made_Object .
  }
  ?nmvw_actor owl:sameAs ?bronbeek_actor .
}
```


# CQ 3

Generate [CQ3_nmvw_obj.csv](CQ3_nmvw_obj.csv)
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT *
WHERE{
  Graph <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
     ?nmvw_acq1 crm:P23_transferred_title_from ?nmvw_actor1 .
     ?nmvw_acq1 crm:P24_transferred_title_of ?nmvw_object .
    
    ?nmvw_acq2 crm:P24_transferred_title_of ?nmvw_object .
    ?nmvw_acq2 crm:P23_transferred_title_from ?nmvw_actor2 .
    
    Filter (?nmvw_actor1!= ?nmvw_actor2)
   }
}
```

In [92]:
import pandas as pd

df1 = pd.read_csv("CQ3_nmvw_obj.csv")
df1 = df1.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
df1 = df1.iloc[: , :-1]
df1.columns = ["nmvw_acq1" , "nmvw_actor1" , "nmvw_object" , "nmvw_acq2" , "nmvw_actor2"]

pairs = set(frozenset(pair) for pair in zip(df1['nmvw_actor1'], df1['nmvw_actor2']))
df1 = pd.DataFrame(pairs, columns=['nmvw_actor1', 'nmvw_actor2'])

print(f"Unique two person's pair in nmvw dataset is: {len(df1)}")

Unique two person's pair in nmvw dataset is: 3878


generate [CQ3_bb_obj.csv](CQ3_bb_obj.csv)

```
SELECT * WHERE{
     ?bb_acq1 crm:P23_transferred_title_from ?bb_actor1 .
     ?bb_acq1 crm:P24_transferred_title_of ?bb_object .
    
    ?bb_acq2 crm:P24_transferred_title_of ?bb_object .
    ?bb_acq2 crm:P23_transferred_title_from ?bb_actor2 .

    Filter (?bb_actor1!= ?bb_actor2)
Graph <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
        ?bb_object a crm:E22_Human-Made_Object.
   }
}
```

In [93]:
import pandas as pd

df2 = pd.read_csv("CQ3_bb_obj.csv")
# df2 = df2.iloc[: , :-1]
df2.columns = [  "bb_acq1" , "bb_actor1" , "bb_object" , "bb_acq2" , "bb_actor2"]

pairs = set(frozenset(pair) for pair in zip(df2['bb_actor1'], df2['bb_actor2']))
df2 = pd.DataFrame(pairs, columns=['bb_actor1', 'bb_actor2'])

print(f"Unique two person's pair in bronbeek dataset is: {len(df2)}")


Unique two person's pair in bronbeek dataset is: 488


Download the match results in [sameAs.csv](sameAs.csv)

SPARQL QUERY:
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT * WHERE{
  ?nmwv_actor owl:sameAs ?bb_actor .
}
```

In [100]:
match_df = pd.read_csv("sameAs.csv")
# match_df = match_df.iloc[:, :-1]
match_df.columns = [ "nmvw_actor" , "bb_actor"]

merged_df1 = df1.merge(match_df, left_on='nmvw_actor1', right_on='nmvw_actor', how='inner')
merged_df2 = df1.merge(match_df, left_on='nmvw_actor2', right_on='nmvw_actor', how='inner')
final_df = pd.concat([merged_df1, merged_df2])
final_df = final_df.drop_duplicates()

result_df1 = final_df.merge(df2, left_on='bb_actor', right_on='bb_actor1', how='inner')
result_df2 = final_df.merge(df2, left_on='bb_actor', right_on='bb_actor2', how='inner')
result_df = pd.concat([result_df2,result_df1])

In [101]:
pair_list = list()

for _, row in result_df.iterrows():
    if row['nmvw_actor1'] == row['nmvw_actor']:
        x1 = row['nmvw_actor1']
        x = row['nmvw_actor2']
    else:
        x = row['nmvw_actor1']

    if row['bb_actor1'] == row['bb_actor']:
        y =  row['bb_actor2']
    else:
        y =  row['bb_actor1']
    
    pair_list.append(frozenset((x,y)))
    try:
        pair_list.append(frozenset((x1,y)))
    except NameError:
        print(f"{row['nmvw_actor1']} never apprears in sameAs")

pair_set = set(pair_list)
pair_df = pd.DataFrame(pair_set, columns=['actor1', 'actor2'])

print(f"The number of new pairs of actor-to-actor connection added is: {len(pair_df)}")

The number of new pairs of actor-to-actor connection added is: 2


In [102]:
all_values = pd.concat([pair_df['actor1'], pair_df['actor2']])

# Get unique values
unique_values = all_values.unique()

# Filter unique values that start with the specified prefix
prefix = 'https://hdl.handle.net/'
filtered_values = [value for value in unique_values if value.startswith(prefix)]

print(f"Unique NMVW actor: {len(filtered_values)}")

Unique NMVW actor: 2


# CQ 4

##### These are the objects, where we do not know it's production place, but do know the acqusuiton actor.
> For 49152 object we did not know the production location

```PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT (COUNT(DISTINCT ?nmvw_obj) AS ?no_object) ?nmvw_actor
WHERE{
 GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
    ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor.
    ?nmvw_acq crm:P24_transferred_title_of ?nmvw_obj .
    ?nmvw_obj crm:P108i_was_produced_by ?nmvw_prod .
    FILTER NOT EXISTS {
      ?nmvw_prod crm:P7_took_place_at ?nmvw_prod_place.}
    }
} GROUP BY ?nmvw_actor ```

##### These are the objects, where we DO KNOW it's production place, and acqusuiton actor.
> For 457722 object, we did not know the production location
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT (COUNT(DISTINCT ?nmvw_obj) AS ?no_object) (COUNT(DISTINCT ?nmvw_prod_place) AS ?no_place) ?nmvw_actor
WHERE{
 GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
     ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor.
      ?nmvw_acq crm:P24_transferred_title_of ?nmvw_obj .
      ?nmvw_obj crm:P108i_was_produced_by ?nmvw_prod .
     ?nmvw_prod crm:P7_took_place_at ?nmvw_prod_place.
    }
} GROUP BY ?nmvw_actor 
```

> If we connect them with acqusistion_actor, for 45756 objects we can prioritise possible location .

In [104]:
import pandas as pd

df1 = pd.read_csv("CQ4_nmvw_object_place_unknown.csv")
df1 = df1.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
df2 = pd.read_csv("CQ4_nmvw_object_place_known.csv")
df2 = df2.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)

In [105]:
merged_df = pd.merge(df1, df2, on='nmvw_actor', how='inner')
print(f"From nmvw dataset, we have {merged_df['no_object_x'].sum()} objects where they are connected to actor whose at least one object has known production location.")

From nmvw dataset, we have 45756 objects where they are connected to actor whose at least one object has known production location.


##### Object connected to Bronbeek actors that as sameAs Links with NMVW

Query to generate [CQ4_bb_object.csv](CQ4_bb_object.csv):

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT (COUNT(DISTINCT ?bb_obj) AS ?no_object) ?bb_actor
WHERE{
    ?nmvw_actor owl:sameAs ?bb_actor .
    {?bb_acq crm:P23_transferred_title_from ?bb_actor .	
    ?bb_acq crm:P24_transferred_title_of ?bb_obj .}
    UNION{
          ?bb_obj crm:P51_has_former_or_current_owner ?bb_actor .
}

 GRAPH <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
    ?bb_obj a crm:E22_Human-Made_Object. }
} GROUP BY ?bb_actor 
```

In [106]:
def cleanup(x: str):
    return int(str(x).replace('"', '').strip())

df3 = pd.read_csv("CQ4_bb_object.csv")
df3 = df3.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
# df3 = df3.iloc[: , :-1]
df3.columns = ["no_object" , "bb_actor"]
df3['no_object'] = df3['no_object'].apply(cleanup)
print(f"From bronbeek dataset, we have {df3['no_object'].sum()} objects that are connected with an actor who have owl:sameAs link to nmvw dataset")

From bronbeek dataset, we have 1030 objects that are connected with an actor who have owl:sameAs link to nmvw dataset


Generate query for [sameAs.csv](sameAs.csv)
```
SELECT * WHERE{
    ?nmvw_acotr owl:sameAs ?bb_actor .
}```

In [107]:
df4 = pd.read_csv("sameAs.csv")
df4 = df4.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
# df4 = df4.iloc[: , :-1]
df4.columns = ['nmvw_actor' , 'bb_actor']

In [108]:
merged_df = pd.merge(df3, df4, on='bb_actor', how='inner')
# when there is multiple match on the bb actor, only kept the first match
merged_df = merged_df.drop_duplicates(subset='bb_actor')
merged_df = pd.merge(merged_df, df2, on='nmvw_actor', how='inner')
merged_df['no_object_x'].sum()

240

In [109]:
print(f"Total actor involved: {merged_df['nmvw_actor'].nunique()}")

Total actor involved: 27


In [18]:
240 + 45756

45996

- **Exact String match:** Now for 0 bronbeek object, we can potentially project possible location.
- **Initial+surname match:** Now for 1841 bronbeek objects, we can potentially project possible location.
- **Surname match:** Now for 12776 bronbeek objects, we can potentially project possible location.
- **Fuzzy String match:** Now for 16503 bronbeek objects, we can potentially project possible location.

# CQ-5

```
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT (AVG(?n) AS ?avg) WHERE
{
  SELECT (COUNT(DISTINCT ?nmvw_obj) AS ?n) 
  WHERE{
      GRAPH <https://pressingmatter.nl/NMVW/ccrdfhiseve.ttl>{
      ?histevet a crm:E7_Activity .
      ?histevet crm:P140i_was_attributed_by ?o .
      ?histevet  crm:P1_is_identified_by ?title .
      ?histevet crm:P4_has_time-span [crm:P82a_begin_of_the_begin ?histevet_btime;
                                       crm:P82b_end_of_the_end ?histevet_etime] .
      ?title crm:P190_has_symbolic_content ?histevet_name .
      ?o a crm:E13_Attribute_Assignment .
      ?o crm:P141_assigned ?nmvw_obj .
    } 
    ?nmvw_obj a crm:E22_Human-Made_Object .
}GROUP BY ?histevet
}
```
> Average object per event: 261.941 with 51 event

Query to download ([CQ5_nmvw.csv](CQ5_nmvw.csv))

```
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?histevet ?histevet_btime ?histevet_etime ?nmvw_obj ?nmvw_actor 
  WHERE{
      GRAPH <https://pressingmatter.nl/NMVW/ccrdfhiseve.ttl>{
      ?histevet a crm:E7_Activity .
      ?histevet crm:P140i_was_attributed_by ?o .
      ?histevet  crm:P1_is_identified_by ?title .
      ?histevet crm:P4_has_time-span [crm:P82a_begin_of_the_begin ?histevet_btime;
                                       crm:P82b_end_of_the_end ?histevet_etime] .
      ?title crm:P190_has_symbolic_content ?histevet_name .
      ?o a crm:E13_Attribute_Assignment .
      ?o crm:P141_assigned ?nmvw_obj .
    } 
    ?nmvw_obj a crm:E22_Human-Made_Object .
    ?nmvw_acq crm:P24_transferred_title_of ?nmvw_obj .
    ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor .	
  	?nmvw_actor owl:sameAs ?bb_actor .
}
```

```
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?histevet ?histevet_btime ?histevet_etime ?nmvw_obj ?nmvw_actor 
  WHERE{
    ?nmvw_actor owl:sameAs ?bb_actor .
    ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor .	
    ?nmvw_acq crm:P24_transferred_title_of ?nmvw_obj .
    ?nmvw_obj a crm:E22_Human-Made_Object .
  GRAPH <https://pressingmatter.nl/NMVW/ccrdfhiseve.ttl>{
    ?o crm:P141_assigned ?nmvw_obj .
    ?o a crm:E13_Attribute_Assignment .
    ?histevet crm:P140i_was_attributed_by ?o .
    ?histevet a crm:E7_Activity .
    ?histevet  crm:P1_is_identified_by ?title .
    ?histevet crm:P4_has_time-span [crm:P82a_begin_of_the_begin ?histevet_btime;
                                       crm:P82b_end_of_the_end ?histevet_etime] .
    ?title crm:P190_has_symbolic_content ?histevet_name .      
    } 
}
```

In [110]:
import pandas as pd

def cleanup_time(x: str):
    # print(type(x))
    return int(str(x).replace('"', '').strip()[:4])

df1 = pd.read_csv("CQ5_nmvw.csv")
df1 = df1.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
df1 = df1.iloc[: , :-1]
df1.columns = [ "histevet" , "histevet_btime" , "histevet_etime" , "nmvw_obj" , "nmvw_actor"]
df1['histevet_btime'] = df1['histevet_btime'].apply(cleanup_time)
df1['histevet_etime'] = df1['histevet_etime'].apply(cleanup_time)

In [111]:
print(f"Total distinct obj:{df1['nmvw_obj'].nunique()} \nTotal distinct actor: {df1['nmvw_actor'].nunique()}\nAvg object: {df1['nmvw_obj'].nunique()/df1['nmvw_actor'].nunique()}")

Total distinct obj:1417 
Total distinct actor: 36
Avg object: 39.361111111111114


In [112]:
# drop the nmvw_obj
df1 = df1.drop(['nmvw_obj'], axis=1)
# Remove duplicate actor to event connections
df1 = df1.drop_duplicates()
df1

,histevet,histevet_btime,histevet_etime,nmvw_actor
0,https://hdl.handle.net/20.500.11840/event31,1935,1936,https://hdl.handle.net/20.500.11840/pi703
64,https://hdl.handle.net/20.500.11840/event11,1906,1906,https://hdl.handle.net/20.500.11840/pi2664
96,https://hdl.handle.net/20.500.11840/event13,1920,1921,https://hdl.handle.net/20.500.11840/pi703
240,https://hdl.handle.net/20.500.11840/event15,1936,1936,https://hdl.handle.net/20.500.11840/pi6890
330,https://hdl.handle.net/20.500.11840/event423,1873,1942,https://hdl.handle.net/20.500.11840/pi21230
331,https://hdl.handle.net/20.500.11840/event423,1873,1942,https://hdl.handle.net/20.500.11840/pi3325
333,https://hdl.handle.net/20.500.11840/event423,1873,1942,https://hdl.handle.net/20.500.11840/pi7750
339,https://hdl.handle.net/20.500.11840/event423,1873,1942,https://hdl.handle.net/20.500.11840/pi56724
383,https://hdl.handle.net/20.500.11840/event423,1873,1942,https://hdl.handle.net/20.500.11840/pi3581
416,https://hdl.handle.net/20.500.11840/event423,1873,1942,https://hdl.handle.net/20.500.11840/pi3153


Query to generate [CQ5_bb.csv](CQ5_bb.csv)

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX aat: <http://vocab.getty.edu/aat/>

SELECT ?bb_obj ?bb_actor ?nmvw_actor ?b_time ?e_time
WHERE{
  	?nmvw_actor owl:sameAs ?bb_actor .
  	?bb_acq2 crm:P23_transferred_title_from ?bb_actor . 
  	?bb_acq2 crm:P24_transferred_title_of ?bb_obj .
    ?prov_activity  crm:P9_consists_of ?bb_acq2 .
    ?prov_activity crm:P2_has_type aat:300055863 .
  	
    ?prov_activity crm:P9_consists_of ?bb_acq .
  	?bb_acq crm:P24_transferred_title_of ?bb_obj .
	  ?bb_acq crm:P4_has_time-span ?time .
  	?time crm:P82a_begin_of_the_begin ?b_time .
    ?time crm:P82b_end_of_the_end ?e_time.
  FILTER (?bb_acq != ?bb_acq2) .

GRAPH <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
    ?bb_obj a crm:E22_Human-Made_Object. }
}
```

In [113]:
def cleanup_time(x: str):
    # Check if x is not None and not an empty string
    try:
        if x == "None" :
            return 0
        if x and str(x).strip():
            # Convert the cleaned string to an integer
            return int(str(x).replace('"', '').strip()[:4])
    except ValueError:
        return 0

# Read the second CSV file into DataFrame df2
df2 = pd.read_csv("CQ5_bb.csv")
df2 = df2.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)

# df2 = df2.iloc[: , :-1]
df2.columns = ["bb_obj" , "bb_actor" , "nmvw_actor" , "b_time" , "e_time"]
df2['b_time'] = df2['b_time'].apply(cleanup_time)
df2['e_time'] = df2['e_time'].apply(cleanup_time)

# Remove duplicate rows in df2
df2 = df2.drop_duplicates() 

# remove bb_actor matched to multiple nmvw_actor
# df2 = df2.drop_duplicates(subset=['bb_obj'])

# Extract unique 'nmvw_actor' values from df2
unique_nmvw_actor = df1['nmvw_actor'].unique()

# Filter df2 to keep only rows where 'nmvw_actor' exists in df2
df2= df2[df2['nmvw_actor'].isin(unique_nmvw_actor)]

df2 = df2.sort_values('bb_actor')

df2

,bb_obj,bb_actor,nmvw_actor,b_time,e_time


In [114]:
print(f"{df2['bb_obj'].nunique()} object has connection with actors where actor has connection with historical event with {df2['nmvw_actor'].nunique()} unique actor")

0 object has connection with actors where actor has connection with historical event with 0 unique actor


In [8]:
35391 + 13359

48750

# CQ-6a

Query to generate [CQ6a_nmvw.csv](CQ6a_nmvw.csv): 

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX aat: <http://vocab.getty.edu/aat/>

SELECT ?nmvw_obj ?nmvw_actor ?b_time ?e_time
WHERE{
 GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
     ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor.
     ?nmvw_acq crm:P24_transferred_title_of ?nmvw_obj .
     ?prov_activity ?p ?nmvw_acq.
     ?prov_activity crm:P2_has_type aat:300055863 .
     ?prov_activity crm:P4_has_time-span ?time .
     ?time crm:P82a_begin_of_the_begin ?b_time .
     ?time crm:P82b_end_of_the_end ?e_time.
    }
}
```

In [1]:
def cleanup_time(x: str):
    return int(str(x).replace('"', '').strip()[-4:])

import pandas as pd

df1 = pd.read_csv("CQ6a_nmvw.csv")
df1 = df1.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
df1 = df1.iloc[: , :-1]

df1.columns = ['nmvw_obj', 'nmvw_actor', 'b_time', 'e_time']
df1['b_time'] = df1['b_time'].apply(cleanup_time)
df1['e_time'] = df1['e_time'].apply(cleanup_time)
df1

,nmvw_obj,nmvw_actor,b_time,e_time
0,https://hdl.handle.net/20.500.11840/128787,https://hdl.handle.net/20.500.11840/pi2338,1972,1972
1,https://hdl.handle.net/20.500.11840/283554,https://hdl.handle.net/20.500.11840/pi3989,1969,1969
2,https://hdl.handle.net/20.500.11840/300252,https://hdl.handle.net/20.500.11840/pi40581,1922,1922
3,https://hdl.handle.net/20.500.11840/143099,https://hdl.handle.net/20.500.11840/pi3683,1978,1978
4,https://hdl.handle.net/20.500.11840/302925,https://hdl.handle.net/20.500.11840/pi38456,2003,2008
...,...,...,...,...
3596,https://hdl.handle.net/20.500.11840/235554,https://hdl.handle.net/20.500.11840/pi38479,2003,2003
3597,https://hdl.handle.net/20.500.11840/235555,https://hdl.handle.net/20.500.11840/pi38479,2003,2003
3598,https://hdl.handle.net/20.500.11840/235556,https://hdl.handle.net/20.500.11840/pi38479,2003,2003
3599,https://hdl.handle.net/20.500.11840/235557,https://hdl.handle.net/20.500.11840/pi38479,2003,2003


In [2]:
grouped_df = df1.groupby(['nmvw_actor', 'b_time', 'e_time'])['nmvw_obj'].nunique().reset_index()
grouped_df = grouped_df.sort_values(by='nmvw_obj', ascending=False)
grouped_df = grouped_df[grouped_df['nmvw_obj'] != 1]
print(f"Number of objects that can be connected with at least one other object \nbased on they are connected by same actor during similar time is: {grouped_df['nmvw_obj'].sum()}")

Number of objects that can be connected with at least one other object 
based on they are connected by same actor during similar time is: 3526


Query to generate: [CQ6a_bb.csv](CQ6a_bb.csv)

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX aat: <http://vocab.getty.edu/aat/>

SELECT ?bb_obj ?bb_actor ?nmvw_actor ?b_time ?e_time
WHERE{
  	?nmvw_actor owl:sameAs ?bb_actor .
  	?bb_acq2 crm:P23_transferred_title_from ?bb_actor . 
  	?bb_acq2 crm:P24_transferred_title_of ?bb_obj .
    ?prov_activity  crm:P9_consists_of ?bb_acq2 .
    ?prov_activity crm:P2_has_type aat:300055863 .
  	
    ?prov_activity crm:P9_consists_of ?bb_acq .
  	?bb_acq crm:P24_transferred_title_of ?bb_obj .
	  ?bb_acq crm:P4_has_time-span ?time .
  	?time crm:P82a_begin_of_the_begin ?b_time .
    ?time crm:P82b_end_of_the_end ?e_time.
  FILTER (?bb_acq != ?bb_acq2) .

GRAPH <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
    ?bb_obj a crm:E22_Human-Made_Object. }
}
```

In [74]:
def cleanup_time(x: str):
    # Check if x is not None and not an empty string
    try:
        if x == "None" :
            return 0
        if x and str(x).strip():
            # Convert the cleaned string to an integer
            return int(str(x).replace('"', '').strip()[:4])
    except ValueError:
        return 0

import pandas as pd

df2 = pd.read_csv("CQ6a_bb.csv")
# df2 = df2.iloc[: , :-1]
df2.columns = ['bb_obj', 'bb_actor', 'nmvw_actor', 'b_time', 'e_time']
df2['b_time'] = df2['b_time'].apply(cleanup_time)
df2['e_time'] = df2['e_time'].apply(cleanup_time)

In [75]:
def merge_with_condition(df1, df2):
    merged_df = pd.merge(df1, df2, on='nmvw_actor', how='inner')
    merged_df['time_diff_b'] = merged_df['b_time_y'] - merged_df['b_time_x']
    merged_df['time_diff_e'] = merged_df['e_time_y'] - merged_df['e_time_x']
    condition = (abs(merged_df['time_diff_b']) <= 20) & (abs(merged_df['time_diff_e']) <= 20)
    return merged_df[condition]

# Merge the dataframes based on the custom condition
result_df = merge_with_condition(df1, df2)
print(result_df.to_csv(index=False))
print("Number of rows in result_df:", len(result_df))

nmvw_obj,nmvw_actor,b_time_x,e_time_x,bb_obj,bb_actor,b_time_y,e_time_y,time_diff_b,time_diff_e
https://hdl.handle.net/20.500.11840/514830,https://hdl.handle.net/20.500.11840/pi46567,2013,2013,https://pressingmatter.nl/Bronbeek/Objects/66596,https://pressingmatter.nl/Bronbeek/Constituents/12755,2015,2015,2,2
https://hdl.handle.net/20.500.11840/514830,https://hdl.handle.net/20.500.11840/pi46567,2013,2013,https://pressingmatter.nl/Bronbeek/Objects/83853,https://pressingmatter.nl/Bronbeek/Constituents/14238,2017,2017,4,4
https://hdl.handle.net/20.500.11840/514830,https://hdl.handle.net/20.500.11840/pi46567,2013,2013,https://pressingmatter.nl/Bronbeek/Objects/11734,https://pressingmatter.nl/Bronbeek/Constituents/2375,1994,1994,-19,-19
https://hdl.handle.net/20.500.11840/514830,https://hdl.handle.net/20.500.11840/pi46567,2013,2013,https://pressingmatter.nl/Bronbeek/Objects/11735,https://pressingmatter.nl/Bronbeek/Constituents/2375,1994,1994,-19,-19
https://hdl.handle.net/20.500.11840/514830

In [76]:
print(f"Unique NMVW actor: {result_df['nmvw_actor'].nunique()}")

Unique NMVW actor: 3


# CQ-6b

Query to generate [CQ6b_nmvw.csv](CQ6b_nmvw.csv): 

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX aat: <http://vocab.getty.edu/aat/>

SELECT ?nmvw_obj ?nmvw_actor ?b_time ?e_time
WHERE{
 GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
     ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor.
     ?nmvw_acq crm:P24_transferred_title_of ?nmvw_obj .
     ?prov_activity ?p ?nmvw_acq.
     ?prov_activity crm:P2_has_type aat:300055863 .
     OPTIONAL{
             ?prov_activity crm:P4_has_time-span ?time .
             ?time crm:P82a_begin_of_the_begin ?b_time .
             ?time crm:P82b_end_of_the_end ?e_time.
     }
    }
}
```

In [77]:
def cleanup_time(x: str):
    try:
        return int(str(x).replace('"', '').strip()[-4:])
    except ValueError:
        return None


import pandas as pd

df1 = pd.read_csv("CQ6b_nmvw.csv")
df1 = df1.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
df1 = df1.iloc[: , :-1]
df1.columns = ['nmvw_obj', 'nmvw_actor', 'b_time', 'e_time']
df1['b_time'] = df1['b_time'].apply(cleanup_time)
df1['e_time'] = df1['e_time'].apply(cleanup_time)
df1

,nmvw_obj,nmvw_actor,b_time,e_time
0,https://hdl.handle.net/20.500.11840/140235,https://hdl.handle.net/20.500.11840/pi2246,NaN,NaN
1,https://hdl.handle.net/20.500.11840/140450,https://hdl.handle.net/20.500.11840/pi2432,NaN,NaN
2,https://hdl.handle.net/20.500.11840/22139,https://hdl.handle.net/20.500.11840/pi7008,NaN,NaN
3,https://hdl.handle.net/20.500.11840/22199,https://hdl.handle.net/20.500.11840/pi2354,NaN,NaN
4,https://hdl.handle.net/20.500.11840/22201,https://hdl.handle.net/20.500.11840/pi7008,NaN,NaN
...,...,...,...,...
528651,https://hdl.handle.net/20.500.11840/21777,https://hdl.handle.net/20.500.11840/pi7008,NaN,NaN
528652,https://hdl.handle.net/20.500.11840/21777,https://hdl.handle.net/20.500.11840/pi39586,NaN,NaN
528653,https://hdl.handle.net/20.500.11840/276706,https://hdl.handle.net/20.500.11840/pi5989,NaN,NaN
528654,https://hdl.handle.net/20.500.11840/240,https://hdl.handle.net/20.500.11840/pi4332,NaN,NaN


In [78]:
df1_time_known = df1.dropna(subset=['b_time', 'e_time'])

grouped_all_df1 = df1.groupby(['nmvw_actor'])['nmvw_obj'].nunique().reset_index()
grouped_all_df1 = grouped_all_df1.sort_values(by='nmvw_obj', ascending=False)

grouped_known_df1 = df1_time_known.groupby(['nmvw_actor'])['nmvw_obj'].nunique().reset_index()
grouped_known_df1 = grouped_known_df1.sort_values(by='nmvw_obj', ascending=False)

merged_df1 = pd.merge(grouped_known_df1, grouped_all_df1, on='nmvw_actor', how='inner')
merged_df1['obj_time_unknown'] = merged_df1['nmvw_obj_y'] - merged_df1['nmvw_obj_x']
print(f"The number of objects we can possibly project the time of acquisition is: {merged_df1['obj_time_unknown'].sum()}")


The number of objects we can possibly project the time of acquisition is: 40186


Query to generate: [CQ6b_bb.csv](CQ6b_bb.csv)

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX aat: <http://vocab.getty.edu/aat/>

SELECT ?bb_obj ?bb_actor ?nmvw_actor ?b_time ?e_time
WHERE{
  	?nmvw_actor owl:sameAs ?bb_actor .
  	?bb_acq2 crm:P23_transferred_title_from ?bb_actor . 
  	?bb_acq2 crm:P24_transferred_title_of ?bb_obj .
    ?prov_activity  crm:P9_consists_of ?bb_acq2 .
    ?prov_activity crm:P2_has_type aat:300055863 .
  	
    ?prov_activity crm:P9_consists_of ?bb_acq .
  	?bb_acq crm:P24_transferred_title_of ?bb_obj .
	  ?bb_acq crm:P4_has_time-span ?time .
  	?time crm:P82a_begin_of_the_begin ?b_time .
    ?time crm:P82b_end_of_the_end ?e_time.
  FILTER (?bb_acq != ?bb_acq2) .

GRAPH <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
    ?bb_obj a crm:E22_Human-Made_Object. }
}
```

In [79]:
def cleanup_time(x: str):
    # Check if x is not None and not an empty string
    try:
        if x == "None" :
            return 0
        if x and str(x).strip():
            # Convert the cleaned string to an integer
            return int(str(x).replace('"', '').strip()[:4])
    except ValueError:
        return 0

import pandas as pd

df2 = pd.read_csv("CQ6b_bb.csv")
# df2 = df2.iloc[: , :-1]
df2.columns = ['bb_obj', 'bb_actor', 'nmvw_actor', 'b_time', 'e_time']
df2['b_time'] = df2['b_time'].apply(cleanup_time)
df2['e_time'] = df2['e_time'].apply(cleanup_time)

In [80]:
filtered_df2 = df2[(df2['b_time'] == 0) & (df2['e_time'] == 0)]

# filtered_df2 = filtered_df2.drop_duplicates(subset=['bb_obj'])

grouped_unknown_df2 = filtered_df2.groupby(['nmvw_actor'])['bb_obj'].nunique().reset_index()
grouped_unknown_df2 = grouped_unknown_df2.sort_values(by='bb_obj', ascending=False)

merged_df2 = pd.merge(grouped_known_df1, grouped_unknown_df2, on='nmvw_actor', how='inner')
print(f"There is {merged_df2['bb_obj'].sum()} objects from bronbeek dataset where don't know the acquisition date \nand from the same collector, we know some acquisition date for nmvw object")

There is 163 objects from bronbeek dataset where don't know the acquisition date 
and from the same collector, we know some acquisition date for nmvw object


> same object can appear multiple times

In [82]:
print(f"Unique NMVW actor: {merged_df2['nmvw_actor'].nunique()}")

Unique NMVW actor: 7


# CQ-6c

Query to generate [CQ6c_nmvw.csv](CQ6c_nmvw.csv): 
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX aat: <http://vocab.getty.edu/aat/>

SELECT ?nmvw_obj ?nmvw_prod_place ?nmvw_actor ?b_time ?e_time
WHERE{
 GRAPH <https://pressingmatter.nl/NMVW/ccrdfobj.ttl>{
     ?nmvw_acq crm:P23_transferred_title_from ?nmvw_actor.
     ?nmvw_acq crm:P24_transferred_title_of ?nmvw_obj .
     ?nmvw_obj crm:P108i_was_produced_by ?nmvw_prod .
     ?nmvw_prod crm:P7_took_place_at ?nmvw_prod_place.
     ?prov_activity ?p ?nmvw_acq.
     ?prov_activity crm:P2_has_type aat:300055863 .
     ?prov_activity crm:P4_has_time-span ?time .
     ?time crm:P82a_begin_of_the_begin ?b_time .
     ?time crm:P82b_end_of_the_end ?e_time.
    }
}
```

In [83]:
def cleanup_time(x: str):
    return int(str(x).replace('"', '').strip()[-4:])

import pandas as pd

df1 = pd.read_csv("CQ6c_nmvw.csv")
df1 = df1.map(lambda x: x.replace('"', '').strip() if isinstance(x, str) else x)
df1 = df1.iloc[: , :-1]
df1.columns = ['nmvw_obj', 'nmvw_prod_place', 'nmvw_actor', 'b_time', 'e_time']
df1['b_time'] = df1['b_time'].apply(cleanup_time)
df1['e_time'] = df1['e_time'].apply(cleanup_time)
df1.head(15)

,nmvw_obj,nmvw_prod_place,nmvw_actor,b_time,e_time
0,https://hdl.handle.net/20.500.11840/128787,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi2338,1972,1972
1,https://hdl.handle.net/20.500.11840/283554,https://hdl.handle.net/20.500.11840/termmaster470,https://hdl.handle.net/20.500.11840/pi3989,1969,1969
2,https://hdl.handle.net/20.500.11840/143099,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi3683,1978,1978
3,https://hdl.handle.net/20.500.11840/302925,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi38456,2003,2008
4,https://hdl.handle.net/20.500.11840/302926,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi38456,2003,2008
5,https://hdl.handle.net/20.500.11840/302927,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi38456,2003,2008
6,https://hdl.handle.net/20.500.11840/302928,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi38456,2003,2008
7,https://hdl.handle.net/20.500.11840/302929,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi38456,2003,2008
8,https://hdl.handle.net/20.500.11840/302930,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi38456,2003,2008
9,https://hdl.handle.net/20.500.11840/302931,https://hdl.handle.net/20.500.11840/termmaster...,https://hdl.handle.net/20.500.11840/pi38456,2003,2008


In [84]:
grouped_df1 = df1.groupby(['nmvw_actor', 'nmvw_prod_place', 'b_time', 'e_time'])['nmvw_obj'].nunique().reset_index()
grouped_df1 = grouped_df1.sort_values(by='nmvw_obj', ascending=False)
grouped_df1 = grouped_df1[grouped_df1['nmvw_obj'] != 1]

print(f"Number of objects that can be connected with at least one other object \nbased on they are connected by same actor during similar time and similar location is: {grouped_df1['nmvw_obj'].sum()}")

Number of objects that can be connected with at least one other object 
based on they are connected by same actor during similar time and similar location is: 3295


Query to generate: [CQ6c_bb.csv](CQ6c_bb.csv)
```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX aat: <http://vocab.getty.edu/aat/>

SELECT ?bb_obj ?bb_actor ?nmvw_actor ?b_time ?e_time
WHERE{
  	?nmvw_actor owl:sameAs ?bb_actor .
  	?bb_acq2 crm:P23_transferred_title_from ?bb_actor . 
  	?bb_acq2 crm:P24_transferred_title_of ?bb_obj .
    ?prov_activity  crm:P9_consists_of ?bb_acq2 .
    ?prov_activity crm:P2_has_type aat:300055863 .
  	
    ?prov_activity crm:P9_consists_of ?bb_acq .
  	?bb_acq crm:P24_transferred_title_of ?bb_obj .
	  ?bb_acq crm:P4_has_time-span ?time .
  	?time crm:P82a_begin_of_the_begin ?b_time .
    ?time crm:P82b_end_of_the_end ?e_time.
  FILTER (?bb_acq != ?bb_acq2) .

GRAPH <https://pressingmatter.nl/Bronbeek/Objects/Objects/assertion/bad21d24/2024-03-01T12:54>{
    ?bb_obj a crm:E22_Human-Made_Object. }
}
```

In [85]:
def cleanup_time(x: str):
    # Check if x is not None and not an empty string
    try:
        if x == "None" :
            return 0
        if x and str(x).strip():
            # Convert the cleaned string to an integer
            return int(str(x).replace('"', '').strip()[:4])
    except ValueError:
        return 0

import pandas as pd

df2 = pd.read_csv("CQ6c_bb.csv")
# df2 = df2.iloc[: , :-1]
df2.columns = ['bb_obj', 'bb_actor', 'nmvw_actor', 'b_time', 'e_time']
df2['b_time'] = df2['b_time'].apply(cleanup_time)
df2['e_time'] = df2['e_time'].apply(cleanup_time)

In [86]:
def merge_with_condition(df1, df2):
    merged_df = pd.merge(df1, df2, on='nmvw_actor', how='inner')
    merged_df['time_diff_b'] = merged_df['b_time_y'] - merged_df['b_time_x']
    merged_df['time_diff_e'] = merged_df['e_time_y'] - merged_df['e_time_x']
    condition = (abs(merged_df['time_diff_b']) <= 20) & (abs(merged_df['time_diff_e']) <= 20)
    return merged_df[condition]

# Merge the dataframes based on the custom condition
result_df = merge_with_condition(df1, df2)
print(result_df.to_csv(index=False))
print("Number of rows in result_df:", len(result_df))

nmvw_obj,nmvw_prod_place,nmvw_actor,b_time_x,e_time_x,bb_obj,bb_actor,b_time_y,e_time_y,time_diff_b,time_diff_e
https://hdl.handle.net/20.500.11840/514830,https://hdl.handle.net/20.500.11840/termmaster7675,https://hdl.handle.net/20.500.11840/pi46567,2013,2013,https://pressingmatter.nl/Bronbeek/Objects/66596,https://pressingmatter.nl/Bronbeek/Constituents/12755,2015,2015,2,2
https://hdl.handle.net/20.500.11840/514830,https://hdl.handle.net/20.500.11840/termmaster7675,https://hdl.handle.net/20.500.11840/pi46567,2013,2013,https://pressingmatter.nl/Bronbeek/Objects/83853,https://pressingmatter.nl/Bronbeek/Constituents/14238,2017,2017,4,4
https://hdl.handle.net/20.500.11840/514830,https://hdl.handle.net/20.500.11840/termmaster7675,https://hdl.handle.net/20.500.11840/pi46567,2013,2013,https://pressingmatter.nl/Bronbeek/Objects/11734,https://pressingmatter.nl/Bronbeek/Constituents/2375,1994,1994,-19,-19
https://hdl.handle.net/20.500.11840/514830,https://hdl.handle.net/20.500.11840/termmaster7

In [87]:
print(f"Unique NMVW actor: {result_df['nmvw_actor'].nunique()}")

Unique NMVW actor: 3
